In [23]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #allows for preprocessing of image data, such as rescaling pixel values

from PIL import Image, ImageEnhance, ImageChops
import cv2
from IPython.display import clear_output
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [12]:
dataset = "C:/Users/ivanl/OneDrive/Desktop/Singapore_Battery_Dataset M"
Alkaline = "C:/Users/ivanl/OneDrive/Desktop/Singapore_Battery_Dataset M/Alkaline (503 imgs)"
Lithium = "C:/Users/ivanl/OneDrive/Desktop/Singapore_Battery_Dataset M/Lithium Ion (324 imgs)"
# to change
NiCd = "C:/Users/ivanl/OneDrive/Desktop/Singapore_Battery_Dataset-main/Nickel-Cadmium"
NiMH = "C:/Users/ivanl/OneDrive/Desktop/Singapore_Battery_Dataset-main/Nickel-MetalHydride"
Valid = "C:/Users/ivanl/OneDrive/Desktop/Singapore_Battery_Dataset-main/validation"

In [24]:
def show_images(folder, limit):
    count = 0
    folder_path = os.path.join(os.getcwd(), folder)
    while count < limit:
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(folder_path, filename)
                image = Image.open(image_path)
                enhancer = ImageEnhance.Contrast(image)
                img_edit1 = enhancer.enhance(2.0)  # Increase contrast
                img_edit1.show()
                count += 1
                if count >= limit:
                    break

In [25]:
def count_images(folder):
    folder_path = os.path.join(os.getcwd(), folder)
    count = 0
    for filename in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, filename)):
            count += 1
    return count

In [33]:
def confirm_images(old_folder, new_folder, limit):
    
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)

    count = 0
    folder_path = os.path.join(os.getcwd(), old_folder)
    while count < limit:
        for filename in os.listdir(folder_path):
            if filename.endswith('.JPG') or filename.endswith('.png') or filename.endswith('.jpeg'):
                image_path = os.path.join(folder_path, filename)
                image = cv2.imread(image_path)
                plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                plt.title("Image")
                plt.axis('off')
                plt.show()
                
                response = input(f"Do you want to keep {filename}? (y/n): ")
                if response.lower() == 'y':
                    clear_output()
                    new_image_path = os.path.join(old_folder, filename)
                    dest_path = os.path.join(new_folder, filename)
                    shutil.move(new_image_path, dest_path)
                    print(f"Saved {filename} to {new_folder}")
                else:
                    clear_output()
                    os.remove(image_path)
                    print(f"Deleted {filename}")
                
                count += 1
                if count >= limit:
                    break

In [ ]:
def crop_images(input_folder, output_folder, limit):
    count = 0
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    while count < limit:
        for filename in os.listdir(input_folder):
            if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
                image_path = os.path.join(input_folder, filename)
                image = Image.open(image_path)
                enhancer = ImageEnhance.Contrast(image)
                img_edit1 = enhancer.enhance(2.0)
                img_np = np.array(img_edit1)

                # Select ROI using OpenCV
                x = cv2.selectROI("Select ROI", img_np, fromCenter=False, showCrosshair=True)
                box = (x[0], x[1], x[0] + x[2], x[1] + x[3])
                imcrop = img_edit1.crop(box)

                output_path = os.path.join(output_folder, filename)
                imcrop.save(output_path)
                count += 1
                clear_output()
                print(f"Cropped image saved to {output_path}")
                if count >= limit:
                    break

In [52]:
def make_copy(folder, limit):
    count = 0
    if not os.path.exists("TEMPFOLDER"):
        os.makedirs("TEMPFOLDER")

    while count < limit:
        for filename in os.listdir(folder):
            image_path = os.path.join(folder, filename)
            image = cv2.imread(image_path)
            
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            plt.title("Image")
            plt.axis('off')
            plt.show()

            response = input(f"Do you want to copy {filename}? (y/n): ")
            
            if response.lower() == 'y':
                x = 0
                no = int(input("How many copies? "))
                
                while x < no:
                    x += 1
                    shutil.copy(image_path, "TEMPFOLDER")
                    new_image_path = os.rename(os.path.join("TEMPFOLDER", filename), os.path.join("TEMPFOLDER", f"{filename}_{x}"))
                    shutil.move(new_image_path, os.path.join(os.getcwd(), folder))
                    if x >= no:
                        break
                
                print(f"Copied {filename} to {folder} {x} times")
                clear_output()
            else:
                clear_output()
                print(f"Skipped {filename}")
            
            count += 1
            if count >= limit:
                break

In [ ]:
img = cv2.imread("C:/Users/ivanl/OneDrive/Desktop/Singapore_Battery_Dataset M/Lithium Ion (324 imgs)/LithiumIon_1.jpg")
white_img = np.full_like(img, 255)
imcrop = cv2.absdiff(img, white_img)
imgray = cv2.cvtColor(imcrop, cv2.COLOR_BGR2GRAY)
ret, thresh1 = cv2.threshold(imgray, 10, 10, cv2.THRESH_BINARY)
contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
imcopy = img.copy()

for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(imcopy, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Use matplotlib to display the image in Jupyter Notebook
plt.imshow(cv2.cvtColor(imcopy, cv2.COLOR_BGR2RGB))
plt.title("Contours")
plt.axis('off')
plt.show()

In [ ]:
img = cv2.imread("C:/Users/ivanl/OneDrive/Desktop/Singapore_Battery_Dataset M/Lithium Ion (324 imgs)/LithiumIon_1.jpg")

img_np = np.array(img)
img = cv2.selectROI("Select ROI", img_np, fromCenter=False, showCrosshair=True)
print(img)

box = (img[0], img[1], img[0] + img[2], img[1] + img[3])
x = Image.open("C:/Users/ivanl/OneDrive/Desktop/Singapore_Battery_Dataset M/Lithium Ion (324 imgs)/LithiumIon_1.jpg")
imcrop = x.crop(box)

imcrop.save("Lithium Ion Cropped.jpg")

plt.imshow(cv2.cvtColor(np.array(imcrop), cv2.COLOR_BGR2RGB))
plt.title("cropped image")
plt.axis('off')
plt.show()

In [ ]:
crop_images("Lithium Ion (324 imgs)", "Lithium Ion Cropped")

Cropped image saved to Lithium Ion Cropped\LithiumIon_1.jpg
Cropped image saved to Lithium Ion Cropped\LithiumIon_100.jpg
Cropped image saved to Lithium Ion Cropped\LithiumIon_101.jpg
Cropped image saved to Lithium Ion Cropped\LithiumIon_102.jpg
Cropped image saved to Lithium Ion Cropped\LithiumIon_103.jpg
Cropped image saved to Lithium Ion Cropped\LithiumIon_104.jpg


In [34]:
confirm_images("Alkaline (503 imgs)", "Alkaline Confirmed", 56)

Deleted Alkaline_460.JPG


KeyboardInterrupt: 

In [44]:
count_images("Alkaline Confirmed")

248

In [51]:
make_copy("Alkaline Confirmed", 248)

Skipped Alkaline_95.png
